# Time series preparation

*****

This notebook creates an example dataset to showcase time-series ideas and methods.

*****


In [1]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [2]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2


# define modules locations (you might have to adapt define_mod_locs.py)
%run swiss_utils/define_mod_locs.py

import os
import shutil

import numpy as np

from datetime import datetime

from swiss_utils.data_cube_utilities.sdc_utilities import load_multi_clean

import datacube
dc = datacube.Datacube()

# silence warning (not recommended during development)
import warnings
warnings.filterwarnings("ignore")

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_utilities import indices_ts_stats

The next cell contains the dataset configuration information:
- product
- geographical extent
- time period
- bands

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool](config_tool.ipynb) notebook,
3. by loading the final cell content of the [config_tool](config_tool.ipynb) notebook using the magic `# %load config_cell.txt`.

In [5]:
# %load "config_cell.txt"
# Configuration

product = 'ls8_lasrc_swiss'
measurements = ['nir', 'green', 'red', 'pixel_qa']

min_lon = 7.132771
max_lon = 7.234418
min_lat = 46.768134
max_lat = 46.813031

start_date = datetime.strptime('2013-03-22', '%Y-%m-%d')
end_date = datetime.strptime('2021-12-30', '%Y-%m-%d')


In [6]:
# Load the dataset and clean it

ds_in, clean_mask = load_multi_clean(dc = dc, products = product,
                                     time = [start_date, end_date],
                                     lon = [min_lon, max_lon], lat = [min_lat, max_lat],
                                     measurements = measurements)
del clean_mask
ds_in = ds_in.where(ds_in >= 0) # keep only positive values
ds_in = ds_in.dropna('time', how='all') # drop scenes without data

In [7]:
# clean_mask
# ds_in

In [8]:
# print(idces_dict)
# print(sm_dict)
ds_in['ndvi'] = (ds_in.nir - ds_in.red) / (ds_in.nir + ds_in.red)
ds_in['ndwi'] = (ds_in.green - ds_in.nir) / (ds_in.green + ds_in.nir)

# 'NDVI': '(',
# 'NDWI': '(ds.green - ds.nir) / (ds.green + ds.nir)',
# 'NDBI': '(ds.swir2 - ds.nir) / (ds.swir2 + ds.nir)'

In [9]:
ds_in.pixel_qa.attrs['flags_definition'] = []
ds_in.ndvi.time.attrs = {}


In [10]:
ds_in.to_netcdf('ls8_lasrc_swiss_fribourg_example.nc')


In [ ]:
# import xarray as xr
# ds_back_in = xr.open_dataset('test_stack_2.nc')